In [1]:
!pip install flask flask-cors transformers torch ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.3 MB/s eta 0:00:00


In [2]:
!pip install pyngrok


In [3]:
from pyngrok import ngrok

# Setup your ngrok authentication
NGROK_AUTH_TOKEN = "2dFoUIhdifBrMnJWtKtqcSmEOiY_4dZaX5VdQJJ9jYygsEBAA"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json
from transformers import pipeline
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

# Load story data (just a simple example)
with open('story.json', 'r') as f:
    story = json.load(f)
print(story)

# Initialize the Hugging Face GPT-Neo model for text generation
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)

@app.route('/story', methods=['POST'])
def continue_story():
    try:
        data = request.json
        current_step = data.get('current_step', 'start')
        decision = data.get('decision', '').lower()
        print(decision)
        print(current_step)
        try:
          if decision:
            print("Getting this step")
            current_step = story[current_step]['choices'][decision]
        except Exception as e:
          return jsonify({'error':str(e)}), 500

        print(decision)
        print(current_step)
        if current_step in story:
            #next_step = story[current_step]['choices'][decision]
            description = story[current_step]['description']
            print(description)
            # Use Hugging Face model to generate rich story content
            generated_text = generator(description, max_length=200, num_return_sequences=1)[0]['generated_text']

            return jsonify({
                'description': generated_text,
                'choices': story[current_step]['choices']
            })
        else:
            return jsonify({
                'error': 'Invalid choice. Please try again.'
            }), 400

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the Flask app
port = 5000
public_url = ngrok.connect(port)  # Create a public URL with ngrok
print(f"Flask app running at {public_url}")

app.run(port=port)


{'start': {'description': 'Choose a charecter : Aurora or Alex', 'choices': {'aurora': 'aurora', 'alex': 'alex'}}, 'alex': {'description': '\n In the heart of the Whispering woods, stood a castle of an all powerful wizard, Aldric the Arcane.\n A wizard soo vastly powerful and evil that the tales of his magic kept even the bravest souls at bay.\n But curiosity can be a powerful force, and for you it was irresistible.\n You wake up in a mysterious forest. What do you do?', 'choices': {'explore': 'explore_forest', 'stay': 'stay_in_place'}}, 'aurora': {'description': '\n In the heart of the Whispering woods, stood a castle of an all powerful wizard, Aldric the Arcane.\n A wizard soo vastly powerful and evil that the tales of his magic kept even the bravest souls at bay.\n But curiosity can be a powerful force, and for you it was irresistible.\n You wake up in a mysterious forest. What do you do?', 'choices': {'explore': 'explore_forest', 'stay': 'stay_in_place'}}, 'explore_forest': {'descr

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Flask app running at NgrokTunnel: "https://055e-34-105-113-227.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
